In [1]:
import sys
!{sys.executable} -m pip install --user torch
import torch
print(torch.__version__)
!{sys.executable} -m pip install --user torchvision
import torchvision
print(torchvision.__version__)


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
2.6.0+cu124

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
0.21.0+cu124


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models  # Import models from torchvision
import os
import numpy as np
import pandas as pd
from tqdm.auto import tqdm  # For progress bars
import wandb
import json


################################################################################
# Model Definition for Part 2 - Using a more sophisticated CNN model
################################################################################
class AdvancedCNN(nn.Module):
    def __init__(self, num_classes=100):
        super(AdvancedCNN, self).__init__()
        # Use ResNet18 model, and we won't use pretrained weights since that's for Part 3
        self.model = models.resnet18(pretrained=False)

        # The first convolution layer
        self.model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)

        # AI: Remove the default max pooling to preserve spatial resolution
        """The original ResNet18 applies a max pooling layer with a kernel size of 3 and a stride of 2 
        immediately after the first convolution, which significantly reduces the spatial resolution. 
        For 32×32 images, this pooling operation may remove too much spatial information. 
        Therefore, replacing it with `nn.Identity()` (i.e., no pooling) 
        helps preserve important details that are critical for classification."""
        self.model.maxpool = nn.Identity()

        # Replace the final fully connected layer for 100 classes
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)


################################################################################
# Define a one epoch training function
################################################################################
def train(epoch, model, trainloader, optimizer, criterion, CONFIG):
    """Train one epoch, e.g. all batches of one epoch."""
    device = CONFIG["device"]
    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    # put the trainloader iterator in a tqdm so it can printprogress
    progress_bar = tqdm(trainloader, desc=f"Epoch {epoch + 1}/{CONFIG['epochs']} [Train]", leave=False)

    # iterate through all batches of one epoch
    for i, (inputs, labels) in enumerate(progress_bar):
        # move inputs and labels to the target device
        inputs, labels = inputs.to(device), labels.to(device)
        
        # TODO - Your code here
        # Clear the gradients
        optimizer.zero_grad()
        # Get the model output
        outputs = model(inputs)
        # Compute loss
        loss = criterion(outputs, labels)
        # Compute gradients
        loss.backward()
        # Update the model parameter
        optimizer.step()
        
        # Accumulate the loss value
        running_loss += loss.item()   ### TODO
        # Get the predicted classes
        _, predicted = outputs.max(1)    ### TODO

        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        progress_bar.set_postfix({"loss": running_loss / (i + 1), "acc": 100. * correct / total})

    train_loss = running_loss / len(trainloader)
    train_acc = 100. * correct / total
    return train_loss, train_acc


################################################################################
# Define a validation function
################################################################################
def validate(model, valloader, criterion, device):
    """Validate the model"""
    model.eval()  # Set to evaluation
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # No need to track gradients

        # Put the valloader iterator in tqdm to print progress
        progress_bar = tqdm(valloader, desc="[Validate]", leave=False)

        # Iterate throught the validation set
        for i, (inputs, labels) in enumerate(progress_bar):
            # move inputs and labels to the target device
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)  ### TODO -- inference
            loss = criterion(outputs, labels)  ### TODO -- loss calculation

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            progress_bar.set_postfix({"loss": running_loss / (i + 1), "acc": 100. * correct / total})

    val_loss = running_loss / len(valloader)
    val_acc = 100. * correct / total
    return val_loss, val_acc


def main():
    ############################################################################
    #    Configuration Dictionary (Modify as needed)
    ############################################################################
    # Updated configuration for Part 2 with appropriate hyperparameters
    CONFIG = {
        "model": "AdvancedCNN",  # Changed model name
        "batch_size": 64,
        "learning_rate": 0.005,  # Adjusted learning rate for the bigger model
        "epochs": 5,  # Increased epochs for better convergence
        "num_workers": 4,
        "device": "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu",
        "data_dir": "./data",
        "ood_dir": "./data/ood-test",
        "wandb_project": "sp25-ds542-challenge",
        "seed": 42,
    }

    import pprint
    print("\nCONFIG Dictionary:")
    pprint.pprint(CONFIG)

    ############################################################################
    #      Data Transformation (Enhanced for Part 2)
    ############################################################################
    # Better model generalization
    # AI: Common improvement methods to improve the diversity of the training data and the generalization ability of the model
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),  # Add padding and random crop
        transforms.RandomHorizontalFlip(),  # Random horizontal flip
        transforms.ToTensor(),
        # CIFAR-100 specific normalization values
        transforms.Normalize((0.5071, 0.4867, 0.4408), 
                             (0.2675, 0.2565, 0.2761)),
    ])

    ###############
    # TODO Add validation and test transforms - NO augmentation for validation/test
    ###############

    # Validation and test transforms (NO augmentation)
    # AI: Standardized parameters for CIFAR-100
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
    ])  ### TODO -- BEGIN SOLUTION

    ############################################################################
    #       Data Loading
    ############################################################################

    trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                             download=True, transform=transform_train)

    # Split train into train and validation (80/20 split)
    train_size = int(0.8 * len(trainset))  ### TODO -- Calculate training set size
    val_size = len(trainset) - train_size   ### TODO -- Calculate validation set size
    trainset, valset = torch.utils.data.random_split(trainset, [train_size, val_size])

    ### TODO -- define loaders and test set
    # Create Dataloader for training data
    trainloader = torch.utils.data.DataLoader(trainset, batch_size = CONFIG["batch_size"], shuffle = True, num_workers = CONFIG["num_workers"])
    # Create DataLoader for validation data
    valloader = torch.utils.data.DataLoader(valset, batch_size = CONFIG["batch_size"], shuffle = False, num_workers = CONFIG["num_workers"])

    # ... (Create validation and test loaders)
    testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size = CONFIG["batch_size"], shuffle = False, num_workers = CONFIG["num_workers"])

    ############################################################################
    #   Instantiate model and move to target device
    ############################################################################
    model = AdvancedCNN(num_classes=100)  # Using advanced model for Part 2
    model = model.to(CONFIG["device"])

    print("\nModel summary:")
    print(f"{model}\n")

    # The following code you can run once to find the batch size that gives you the fastest throughput.
    # You only have to do this once for each machine you use, then you can just
    # set it in CONFIG.
    SEARCH_BATCH_SIZES = False
    if SEARCH_BATCH_SIZES:
        from utils import find_optimal_batch_size
        print("Finding optimal batch size...")
        optimal_batch_size = find_optimal_batch_size(model, trainset, CONFIG["device"], CONFIG["num_workers"])
        CONFIG["batch_size"] = optimal_batch_size
        print(f"Using batch size: {CONFIG['batch_size']}")

    ############################################################################
    # Loss Function, Optimizer and optional learning rate scheduler
    ############################################################################
    # Use cross-entropy for classification
    # AI: For CIFAR-100, it's better to choose nn.CrossEntropyLoss
    criterion = nn.CrossEntropyLoss()   ### TODO -- define loss criterion
    # Use SGD with momentum and weight decay
    # AI: For CIFAR-100, it's better to choose SGD
    optimizer = optim.SGD(model.parameters(), lr=CONFIG["learning_rate"], momentum=0.9, weight_decay = 5e-4)   ### TODO -- define optimizer
    # Improved learning rate scheduler(from StepLR to CosineAnnealingLR)
    """CosineAnnealingLR offers a smooth, continuous decay of the learning rate instead of the sudden drops seen with StepLR. 
    This gradual change results in more stable training during later stages, 
    leading to better convergence and improved final performance."""
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CONFIG["epochs"])  # Add a scheduler   ### TODO -- you can optionally add a LR scheduler

    # Initialize wandb
    wandb.init(project="-sp25-ds542-challenge", config=CONFIG)
    wandb.watch(model)  # watch the model gradients

    ############################################################################
    # --- Training Loop ---
    ############################################################################
    best_val_acc = 0.0

    for epoch in range(CONFIG["epochs"]):
        train_loss, train_acc = train(epoch, model, trainloader, optimizer, criterion, CONFIG)
        val_loss, val_acc = validate(model, valloader, criterion, CONFIG["device"])
        scheduler.step()

        # log to WandB
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc,
            "lr": optimizer.param_groups[0]["lr"]  # Log learning rate
        })

        # Save the best model (based on validation accuracy)
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_model.pth")
            wandb.save("best_model.pth")

    wandb.finish()

    ############################################################################
    # Evaluation -- shouldn't have to change the following code
    ############################################################################
    import eval_cifar100
    import eval_ood

    # --- Evaluation on Clean CIFAR-100 Test Set ---
    predictions, clean_accuracy = eval_cifar100.evaluate_cifar100_test(model, testloader, CONFIG["device"])
    print(f"Clean CIFAR-100 Test Accuracy: {clean_accuracy:.2f}%")

    # --- Evaluation on OOD ---
    all_predictions = eval_ood.evaluate_ood_test(model, CONFIG)

    # --- Create Submission File (OOD) ---
    submission_df_ood = eval_ood.create_ood_df(all_predictions)
    submission_df_ood.to_csv("submission_ood_part2.csv", index=False)
    print("submission_ood_part2.csv created successfully.")


if __name__ == '__main__':
    main()



CONFIG Dictionary:
{'batch_size': 64,
 'data_dir': './data',
 'device': 'cuda',
 'epochs': 5,
 'learning_rate': 0.005,
 'model': 'AdvancedCNN',
 'num_workers': 4,
 'ood_dir': './data/ood-test',
 'seed': 42,
 'wandb_project': 'sp25-ds542-challenge'}


/usr4/dl4ds/shiychen/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr4/dl4ds/shiychen/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)



Model summary:
AdvancedCNN(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): Identity()
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shiychen (shiychen-boston-university). Use `wandb login --relogin` to force relogin


Epoch 1/5 [Train]:   0%|          | 0/625 [00:00<?, ?it/s]

...

distortion18 (Severity 3)


Evaluating distortion18 (Severity 4):   0%|          | 0/157 [00:00<?, ?it/s]

distortion18 (Severity 4)


Evaluating distortion18 (Severity 5):   0%|          | 0/157 [00:00<?, ?it/s]

distortion18 (Severity 5)
submission_ood_part2.csv created successfully.
